# TSA Chapter 0: Seasonality Modeling

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch0/TSA_ch0_seasonal/TSA_ch0_seasonal.ipynb)

This notebook demonstrates seasonality modeling:
- Seasonal pattern visualization and seasonal indices
- Dummy variables vs Fourier terms for modeling seasonality
- Seasonal adjustment methods

In [ ]:
!pip install matplotlib numpy scipy pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 10,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'legend.fontsize': 8,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'lines.linewidth': 1.5,
    'axes.prop_cycle': plt.cycler('color', list(COLORS.values())),
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

CHARTS_DIR = os.path.join(os.path.dirname(os.path.abspath('.')), '..', '..', 'charts')

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    # Also save to main charts directory for the lecture
    try:
        charts_path = os.path.join(CHARTS_DIR, name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass  # Skip if running on Colab without the charts dir
    print(f'Saved: {name}.pdf + .png')

def add_legend_below(ax, ncol=3):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=ncol, frameon=False)

In [ ]:
# Chart: seasonal_pattern
# Seasonal indices and monthly pattern from real US retail sales
np.random.seed(42)
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

try:
    ret_sp_df = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?id=RSXFS', parse_dates=['DATE'])
    ret_sp_df['RSXFS'] = pd.to_numeric(ret_sp_df['RSXFS'], errors='coerce')
    ret_sp_df = ret_sp_df.dropna()
    ret_sp_df['month'] = ret_sp_df['DATE'].dt.month
    ret_sp_df['year'] = ret_sp_df['DATE'].dt.year
    # Compute seasonal indices from last 5 complete years
    recent = ret_sp_df[ret_sp_df['year'].isin(ret_sp_df['year'].unique()[-6:-1])]
    monthly_means = recent.groupby('month')['RSXFS'].mean()
    overall_mean = monthly_means.mean()
    seasonal_idx = (monthly_means / overall_mean).values.tolist()
    # Get 3 years for panel 2
    years_data = {}
    for i, yr in enumerate(ret_sp_df['year'].unique()[-4:-1]):
        yr_data = ret_sp_df[ret_sp_df['year'] == yr]['RSXFS'].values
        if len(yr_data) == 12:
            years_data[f'{yr}'] = yr_data
    print(f'Retail sales loaded, computed seasonal indices')
except Exception:
    seasonal_idx = [0.85, 0.82, 0.90, 0.95, 1.05, 1.15, 1.25, 1.22, 1.05, 0.95, 0.88, 0.93]
    years_data = {}
    for yr_offset, yr_name in enumerate(['Year 1', 'Year 2', 'Year 3']):
        base = 100 + yr_offset * 20
        years_data[yr_name] = [base * s + np.random.randn() * 3 for s in seasonal_idx]

colors_bar = [COLORS['red'] if s < 1 else COLORS['green'] for s in seasonal_idx]
axes[0].bar(months, seasonal_idx, color=colors_bar, alpha=0.7, edgecolor='white')
axes[0].axhline(y=1, color='black', linewidth=1, linestyle='-')
axes[0].set_title('Seasonal Indices (US Retail Sales)', fontweight='bold')
axes[0].set_ylabel('$S_t$ (ratio)')
axes[0].set_ylim(min(seasonal_idx) - 0.05, max(seasonal_idx) + 0.05)
axes[0].tick_params(axis='x', rotation=45)

# Monthly pattern across years
colors_yr = [COLORS['blue'], COLORS['green'], COLORS['red']]
for i, (yr_name, yr_vals) in enumerate(years_data.items()):
    if i < 3:
        axes[1].plot(months[:len(yr_vals)], yr_vals, color=colors_yr[i], marker='o', markersize=4,
                     linewidth=1.2, label=yr_name)
axes[1].set_title('Seasonal Pattern Across Years', fontweight='bold')
axes[1].set_ylabel('Sales ($ millions)')
axes[1].tick_params(axis='x', rotation=45)
add_legend_below(axes[1], ncol=3)

fig.tight_layout(rect=[0, 0.02, 1, 1])
save_chart(fig, 'seasonal_pattern')
plt.show()

In [ ]:
# Chart: seasonality_fourier_dummies
# Comparison of dummy variables and Fourier terms for real seasonal pattern
np.random.seed(42)
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

months_num = np.arange(1, 13)
month_names = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

# Use seasonal indices computed in previous cell (or fallback)
try:
    true_pattern = seasonal_idx
except NameError:
    true_pattern = [0.85, 0.82, 0.90, 0.95, 1.05, 1.15, 1.25, 1.22, 1.05, 0.95, 0.88, 0.93]

# Dummy fit (exact)
axes[0].bar(months_num, true_pattern, color=COLORS['blue'], alpha=0.7, edgecolor='white')
axes[0].axhline(y=1, color='black', linewidth=0.5, linestyle=':')
axes[0].set_title('Dummy Variables\n(captures any pattern)', fontweight='bold')
axes[0].set_xticks(months_num)
axes[0].set_xticklabels(month_names, fontsize=8)
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Seasonal Factor')

# Fourier fit
t_fine = np.linspace(1, 12, 100)
axes[1].scatter(months_num, true_pattern, color=COLORS['blue'], s=50, zorder=5, label='Data')

for K, c, ls in [(1, COLORS['green'], '--'), (2, COLORS['orange'], '-.'), (6, COLORS['red'], '-')]:
    fourier_fit = np.ones_like(t_fine) * np.mean(true_pattern)
    for k in range(1, K + 1):
        a_k = (2/12) * sum(true_pattern[j-1] * np.sin(2*np.pi*k*j/12) for j in range(1, 13))
        b_k = (2/12) * sum(true_pattern[j-1] * np.cos(2*np.pi*k*j/12) for j in range(1, 13))
        fourier_fit += a_k * np.sin(2*np.pi*k*t_fine/12) + b_k * np.cos(2*np.pi*k*t_fine/12)
    axes[1].plot(t_fine, fourier_fit, color=c, linewidth=1.5, linestyle=ls, label=f'K={K}')

axes[1].axhline(y=1, color='black', linewidth=0.5, linestyle=':')
axes[1].set_title('Fourier Terms\n(sinusoidal approximation)', fontweight='bold')
axes[1].set_xticks(months_num)
axes[1].set_xticklabels(month_names, fontsize=8)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Seasonal Factor')
add_legend_below(axes[1], ncol=4)

fig.tight_layout(rect=[0, 0.02, 1, 1])
save_chart(fig, 'seasonality_fourier_dummies')
plt.show()

In [ ]:
# Chart: seasonal_adjustment
# Before and after seasonal adjustment on real US retail sales
np.random.seed(42)
fig, axes = plt.subplots(2, 1, figsize=(8, 5), sharex=True)

try:
    ret_sa_df = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?id=RSXFS', parse_dates=['DATE'])
    ret_sa_df['RSXFS'] = pd.to_numeric(ret_sa_df['RSXFS'], errors='coerce')
    ret_sa_df = ret_sa_df.dropna().tail(120)
    t = np.arange(len(ret_sa_df))
    y = ret_sa_df['RSXFS'].values.astype(float)
    # Compute MA(12) trend
    kernel = np.ones(12) / 12
    trend_ma = np.convolve(y, kernel, mode='same')
    # Compute seasonal component via ratio-to-moving-average
    ratio = y / np.where(trend_ma > 0, trend_ma, 1)
    months_arr = ret_sa_df['DATE'].dt.month.values
    seasonal_factors = np.zeros(len(y))
    for m in range(1, 13):
        mask = months_arr == m
        seasonal_factors[mask] = np.mean(ratio[mask])
    adjusted = y / seasonal_factors
    data_label = 'US Retail Sales'
except Exception:
    t = np.arange(120)
    trend_sa = 100 + 0.5 * t
    seasonal_sa = 15 * np.sin(2 * np.pi * t / 12)
    y = trend_sa + seasonal_sa + np.random.randn(120) * 2
    trend_ma = trend_sa
    adjusted = y - seasonal_sa
    data_label = 'Original'

axes[0].plot(t, y, color=COLORS['blue'], linewidth=1.2, label=data_label)
axes[0].plot(t, trend_ma, color=COLORS['red'], linewidth=1.5, linestyle='--', label='Trend (MA-12)')
axes[0].set_title('Original Series with Seasonality', fontweight='bold')
axes[0].set_ylabel('$X_t$')
add_legend_below(axes[0], ncol=2)

axes[1].plot(t, adjusted, color=COLORS['green'], linewidth=1.2, label='Seasonally Adjusted')
axes[1].plot(t, trend_ma, color=COLORS['red'], linewidth=1.5, linestyle='--', label='Trend (MA-12)')
axes[1].set_title('Seasonally Adjusted Series', fontweight='bold')
axes[1].set_xlabel('Time (months)')
axes[1].set_ylabel('$X_t^{adj}$')
add_legend_below(axes[1], ncol=2)

fig.tight_layout(rect=[0, 0.02, 1, 1])
save_chart(fig, 'seasonal_adjustment')
plt.show()